# Marine Invertebrates Classification Challenge by #ZindiWeekendz

File name: InvertebrateAI.ipynb

Author: kogni7

Date: August 2021

## Contents

* 1 Preparation
* 2 Data
* 3 Training
* 4 Prediction and Submission

This notebook uses only the data sets provided by ZINDI. These data sets contain images of deep sea invertebrates. These are the only used features in this notebook. The task is to classify the images.

The file system for this project is:

* InvertebrateAI (root)
    * InvertebrateAI.ipynb (this notebook)
    * Data
        * train_small
        * test_small
        * SampleSubmission_.csv
    * Submission
        * 1 - x: Submission directories named by the version number
            * submission.csv

This jupyter notebook runs in Google Colab without special configuration. GPU is enabled.

This notebook uses a Transfer learning based approach.

Reference: https://github.com/kogni7git/solutions/blob/main/PAYGoAI.ipynb


## 1 Preparation

In [1]:
import time
start_time = time.time()

### Installation, Libraries and Seed

In [2]:
# Installation
!pip install efficientnet_pytorch
!pip install torchtoolbox

# Seed, Libraries
SEED = 42

# Math
import numpy as np
print("Numpy Version: " + str(np.__version__))

import random
import os
os.environ['PYTHONHASHSEED'] = str(SEED)

np.random.seed(SEED)

random.seed(SEED)

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
print("PyTorch Version: " + str(torch.__version__))
torch.manual_seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = False

import torchvision
from torchvision import datasets, transforms
print("PyTorch torchvision Version: " + str(torchvision.__version__))

# Mix-Up
from torchtoolbox.tools import mixup_data, mixup_criterion

# EfficientNet
import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
print("efficientnet_pytorch Version: " + str(efficientnet_pytorch.__version__))

# CSV
import pandas as pd
print("Pandas Version: " + str(pd.__version__))

# Machine Learning
import sklearn
from sklearn.model_selection import KFold
print("SciKit-Learn Version: " + str(sklearn.__version__))

# Images
import PIL
from PIL import Image
print("PIL Version: " + str(PIL.__version__))

from tqdm import tqdm
import gc

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=08037c470feef930e163d0fc7139091d36964c2fe9ef07e5226f89191927cd03
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 58 kB 2.7 MB/s 
Numpy Version: 1.19.5
PyTorch Version: 1.9.0+cu102
PyTorch torchvision Version: 0.10.0+cu102
efficientnet_pytorch Version: 0.7.1
Pandas Version: 1.1.5
SciKit-Learn Version: 0.22.2.post1
PIL Version: 7.1.2


In [3]:
!nvidia-smi

Sun Oct  3 21:03:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Parameters

In [4]:
CLASSES = 137

EPOCHS = 10

BATCH_SIZE = 32

TEST_BATCH_SIZE = 128

"""
b0: 224
b1: 240
"""
NET = 'efficientnet-b1'

IMAGE_SIZE = 240

CV = 5

VERBOSE = True

LEARNING_RATE = 0.5e-4
SCHEDULER = False

alpha = 0.5
MIX_UP = False

# The Version
VERSION = "1"

# for use in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
WD = os.getcwd() + "/drive/My Drive/InvertebrateAI"
print(WD)

/content/drive/My Drive/InvertebrateAI


## 2 Data

In [6]:
# images for training
transformations = transforms.Compose([transforms.RandomResizedCrop(IMAGE_SIZE), transforms.ToTensor()])
images_train = datasets.ImageFolder(WD + "/Data/train_small", transform=transformations)

In [7]:
sample_submission_csv = pd.read_csv(WD + "/Data/SampleSubmission_.csv")
sample_submission_csv.head()

,FILE,Pteraster_capensis,Porifera,Astropecten_irregularis_pontoporeus,Terebratulina_sp_,Neolithodes_asperrimus,Munida_benguela,Cypraeovula_iutsui,Kaloplocamus_ramosus,Actinoscyphia_plebeia,Echinus_gilchristi,Synallactes_viridilimus,Eleutherobia_variable,Neopilumnoplax_heterochir,Pleurobranchaea_bubala,Pagurus_cuanensis,Parapagurus_bouvieri,Inachidae,Pseudarchaster_tessellatus,Pasiphaea_sp._1,Suberites_dandelenae,Comitas_saldanhae,Poraniopsis_echinaster,Pterygosquilla_capensis,Holothuroidea,Exodromidia_spinosa,Aristeus_varidens,Philinopsis_capensis,Vitjazmaia_latidactyla,Perissasterias_polyacantha,Ascidiacea,Toraster_tuberculatus,Nudibranchia,Mediaster_bairdi_capensis,Limopsis_chuni,Comitas_stolida,Psilaster_acuminatus,Lithodes_ferox,Hermit_crab,Solenocera_africana,...,Ophiura_costata_costata,Pelagia_noctiluca,Sclerasterias_spp,Stylasteridae,Fusivoluta_pyrrhostoma,Actinoptilum_molle,Pseudarchaster_brachyactis,Nassarius speciosus,Chondraster_elattosis,Ophiothrix_aristulata,Aphelodoris_sp_,Fusinus_africanae,Scaphander_punctostriatus,Polychaete_tubes_(only),Polychaete_worms,Polyechinus_agulhensis,Chrysaora_spp,Projasus_parkeri,Lamellaria_Coriocella_spp,Hippasteria_phrygiana,Mursia_cristiata,Prawns,Nassarius_vinctus,Marthasterias_africana,Goneplax_clevai,Alcyonacea,Athleta_lutosa,Pseudodromia_spp_,Rochinia_hertwigi,Africolaria_rutila,Neptuneopsis_gilchristi,Pseudodromia_rotunda,Pecten_sulcicostatus,Plesionika_martia,Rossella_antarctica,Philine_aperta,Mycale_anisochela,Henricia_abyssalis,Triviella_spp_,Cheilostomatida
0,FNDG3L9.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0RASRVM.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,JMXLWAX.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,CV8FWT2.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,V34APYI.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# images for prediction
path = WD + "/Data/test_small/"
images = sample_submission_csv.FILE

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, images):
        self.images = images

    def __getitem__(self, idx):
        return transformations(Image.open(path + self.images[idx]))

    def __len__(self):
        return len(self.images)

This is an efficient net with a linear layer and sigmoidal activation.

In [9]:
class EfficientNeuralNet(nn.Module):
    def __init__(self, net):
        super(EfficientNeuralNet, self).__init__()
        self.net = EfficientNet.from_pretrained(net)
        self.ffn1 = nn.Linear(self.net._fc.out_features, 128)
        self.ffn2 = nn.Linear(128, CLASSES)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.net(x)
        x = self.ffn1(x)
        x = self.relu(x)
        return self.sig(self.ffn2(x))

## 3 Training

In [10]:
def training(data_loader, model, optimizer):

    model.train()

    losses = []

    for data in tqdm(data_loader):
        image = data[0].cuda() / 255
        target = data[1].cuda()

        optimizer.zero_grad()

        if MIX_UP:
            image, target_a, target_b, lam = mixup_data(image, target, alpha)

        output = model(image.float())

        if MIX_UP:
            loss = mixup_criterion(nn.CrossEntropyLoss(), output.squeeze().float(), target_a, target_b, lam)
        else:
            loss = nn.CrossEntropyLoss()(output.squeeze().float(), target)

        loss.backward()
        optimizer.step()

        losses.append(loss.data.cpu())

        if MIX_UP:
            del target_a, target_b, lam

        del output, loss, image, target
        gc.collect()

    return np.mean(losses)


def evaluation(data_loader, model):
    
    model.eval()
    
    losses = []

    with torch.no_grad():
        for data in tqdm(data_loader):
            image = data[0].cuda() / 255
            target = data[1].cuda()

            output = model(image.float())
            loss = nn.CrossEntropyLoss()(output.squeeze().float(), target)

            losses.append(loss.data.cpu())

            del output, loss, image, target
            gc.collect()

    return np.mean(losses)


def prediction(data_loader, model):
    model.eval()
    
    outputs = []

    with torch.no_grad():
        for data in tqdm(data_loader):
            image = data.cuda() / 255

            output = model(image.float())
            outputs.append(output.squeeze().data.cpu())

            del output, image
            gc.collect()

        return torch.cat(outputs)

In [11]:
def seed_worker(worker_id):
    """
    https://pytorch.org/docs/stable/notes/randomness.html
    """
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


cv = 1

STATISTICS = {}
STATISTICS['TRAIN'] = np.zeros((EPOCHS, CV))
STATISTICS['VALIDATION'] = np.zeros((EPOCHS, CV))

# Reference for Cross-Validation: https://www.machinecurve.com/index.php/2021/02/03/how-to-use-k-fold-cross-validation-with-pytorch/
for train, val in KFold(n_splits=CV, shuffle=True, random_state=SEED).split(images_train):
    print("Run {} of {}.".format(cv, CV))

    g = torch.Generator()
    g.manual_seed(SEED)
    train_subsampler = torch.utils.data.SubsetRandomSampler(train, generator=g)

    g = torch.Generator()
    g.manual_seed(SEED)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val, generator=g)

    # Data
    g = torch.Generator()
    g.manual_seed(SEED)
    train_data_loader = torch.utils.data.DataLoader(images_train, batch_size=BATCH_SIZE, sampler=train_subsampler, worker_init_fn=seed_worker, generator=g, num_workers=2)

    g = torch.Generator()
    g.manual_seed(SEED)
    val_data_loader = torch.utils.data.DataLoader(images_train, batch_size=TEST_BATCH_SIZE, sampler=val_subsampler, worker_init_fn=seed_worker, generator=g, num_workers=2)

    # Model
    model = EfficientNeuralNet(NET)
    model = model.cuda()

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    if SCHEDULER:
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

    BEST_LogLoss = np.inf

    for epoch in range(EPOCHS):
        train_loss = training(train_data_loader, model, optimizer)
        val_loss = evaluation(val_data_loader, model)

        if SCHEDULER:
            scheduler.step()

        if BEST_LogLoss > val_loss:
            torch.save(model.state_dict(), '/content/model_' + str(cv) + '.pt')
            BEST_LogLoss = val_loss

        if VERBOSE:
            print("Epoch: {}; TRAINING: {:.3f}; VALIDATION: {:.3f}".format(epoch, train_loss, val_loss))

        STATISTICS['TRAIN'][epoch, cv-1] = train_loss
        STATISTICS['VALIDATION'][epoch, cv-1] = val_loss

    del BEST_LogLoss, train_loss, val_loss, model, train_subsampler, train_data_loader, val_subsampler, val_data_loader
    gc.collect()
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

    print("\n")
    cv += 1

print("Result:")
for epoch in range(EPOCHS):
    print("Epoch: {}; TRAINING: {:.3f}; VALIDATION: {:.3f}".format(epoch, np.mean(STATISTICS['TRAIN'][epoch, :]), np.mean(STATISTICS['VALIDATION'][epoch, :])))

Run 1 of 5.


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b1-f1951068.pth


  0%|          | 0.00/30.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b1


100%|██████████| 5/5 [04:27<00:00, 53.49s/it]


...

Epoch: 7; TRAINING: 4.213; VALIDATION: 4.442


100%|██████████| 5/5 [00:06<00:00,  1.30s/it]


Epoch: 8; TRAINING: 4.177; VALIDATION: 4.302


100%|██████████| 5/5 [00:06<00:00,  1.29s/it]


Epoch: 9; TRAINING: 4.147; VALIDATION: 4.205


Result:
Epoch: 0; TRAINING: 4.896; VALIDATION: 4.920
Epoch: 1; TRAINING: 4.803; VALIDATION: 4.920
Epoch: 2; TRAINING: 4.673; VALIDATION: 4.920
Epoch: 3; TRAINING: 4.541; VALIDATION: 4.919
Epoch: 4; TRAINING: 4.417; VALIDATION: 4.889
Epoch: 5; TRAINING: 4.324; VALIDATION: 4.783
Epoch: 6; TRAINING: 4.256; VALIDATION: 4.612
Epoch: 7; TRAINING: 4.209; VALIDATION: 4.437
Epoch: 8; TRAINING: 4.173; VALIDATION: 4.321
Epoch: 9; TRAINING: 4.144; VALIDATION: 4.227


## 4 Prediction and Submission

In [12]:
# We must use the order of sample_submission_csv instead of an alphabetical order.
classes = images_train.class_to_idx
columns = list(sample_submission_csv.columns)[1:]

# Reference for next 2 lines: https://stackoverflow.com/questions/21773866/how-to-sort-a-dictionary-based-on-a-list-in-python
sample_submission_csv_dict = {key: i for i, key in enumerate(columns)}
order = sorted(classes.items(), key=lambda pair:sample_submission_csv_dict[pair[0]])
indices = [order[i][1] for i in range(len(order))]

In [13]:
predictions = torch.zeros(len(sample_submission_csv), CLASSES)

for cv in range(CV):
    # Data
    g = torch.Generator()
    g.manual_seed(SEED)
    TEST_SET = TestDataset(images)
    test_data_loader = torch.utils.data.DataLoader(TEST_SET, batch_size = TEST_BATCH_SIZE, worker_init_fn=seed_worker, generator=g, num_workers=2)

    # Model
    model = EfficientNeuralNet(NET)
    model.load_state_dict(torch.load('/content/model_' + str(cv + 1) + '.pt'))
    model = model.cuda()

    # Prediction
    outputs = prediction(test_data_loader, model)
    predictions += outputs

    del model, TEST_SET, test_data_loader, outputs

    gc.collect()
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

    predictions /= CV
    sample_submission_csv.iloc[:, 1:] = torch.index_select(predictions, 1, torch.tensor(indices))

os.mkdir(WD + '/Submission/' + str(VERSION)) 
sample_submission_csv.to_csv(WD + '/Submission/' + str(VERSION) + '/submission.csv', index=False)

Loaded pretrained weights for efficientnet-b1


100%|██████████| 12/12 [09:17<00:00, 46.44s/it]


Loaded pretrained weights for efficientnet-b1


100%|██████████| 12/12 [00:12<00:00,  1.07s/it]


Loaded pretrained weights for efficientnet-b1


100%|██████████| 12/12 [00:12<00:00,  1.07s/it]


Loaded pretrained weights for efficientnet-b1


100%|██████████| 12/12 [00:12<00:00,  1.08s/it]


Loaded pretrained weights for efficientnet-b1


100%|██████████| 12/12 [00:12<00:00,  1.08s/it]


In [14]:
sample_submission_csv

,FILE,Pteraster_capensis,Porifera,Astropecten_irregularis_pontoporeus,Terebratulina_sp_,Neolithodes_asperrimus,Munida_benguela,Cypraeovula_iutsui,Kaloplocamus_ramosus,Actinoscyphia_plebeia,Echinus_gilchristi,Synallactes_viridilimus,Eleutherobia_variable,Neopilumnoplax_heterochir,Pleurobranchaea_bubala,Pagurus_cuanensis,Parapagurus_bouvieri,Inachidae,Pseudarchaster_tessellatus,Pasiphaea_sp._1,Suberites_dandelenae,Comitas_saldanhae,Poraniopsis_echinaster,Pterygosquilla_capensis,Holothuroidea,Exodromidia_spinosa,Aristeus_varidens,Philinopsis_capensis,Vitjazmaia_latidactyla,Perissasterias_polyacantha,Ascidiacea,Toraster_tuberculatus,Nudibranchia,Mediaster_bairdi_capensis,Limopsis_chuni,Comitas_stolida,Psilaster_acuminatus,Lithodes_ferox,Hermit_crab,Solenocera_africana,...,Ophiura_costata_costata,Pelagia_noctiluca,Sclerasterias_spp,Stylasteridae,Fusivoluta_pyrrhostoma,Actinoptilum_molle,Pseudarchaster_brachyactis,Nassarius speciosus,Chondraster_elattosis,Ophiothrix_aristulata,Aphelodoris_sp_,Fusinus_africanae,Scaphander_punctostriatus,Polychaete_tubes_(only),Polychaete_worms,Polyechinus_agulhensis,Chrysaora_spp,Projasus_parkeri,Lamellaria_Coriocella_spp,Hippasteria_phrygiana,Mursia_cristiata,Prawns,Nassarius_vinctus,Marthasterias_africana,Goneplax_clevai,Alcyonacea,Athleta_lutosa,Pseudodromia_spp_,Rochinia_hertwigi,Africolaria_rutila,Neptuneopsis_gilchristi,Pseudodromia_rotunda,Pecten_sulcicostatus,Plesionika_martia,Rossella_antarctica,Philine_aperta,Mycale_anisochela,Henricia_abyssalis,Triviella_spp_,Cheilostomatida
0,FNDG3L9.jpeg,0.249517,0.213903,0.030718,0.001101,0.004696,0.007740,0.003627,0.015765,0.011668,0.080681,0.000754,0.003635,0.000432,0.168747,0.005099,0.001255,0.000107,0.031553,0.000170,0.007085,0.001268,0.014753,0.000889,0.000390,0.000151,0.017144,0.118255,0.003542,0.004703,0.106318,0.246295,0.000689,0.075934,0.005836,0.002066,0.004801,0.000867,0.004722,0.008419,...,0.067296,0.005597,0.005343,0.002250,0.007737,0.008930,0.057677,0.015629,0.174614,0.001553,0.001018,0.007043,0.001354,0.000176,0.005431,0.024917,0.003638,0.000537,0.008886,0.016164,0.003990,0.003806,0.001220,0.000085,0.011041,0.045214,0.003020,0.010357,0.000324,0.000664,0.000304,0.001276,0.010278,0.000233,0.003981,0.001831,0.005870,0.039728,0.000951,0.000237
1,0RASRVM.jpeg,0.248185,0.220635,0.012700,0.003110,0.000696,0.009705,0.003798,0.004873,0.008523,0.150040,0.001165,0.012905,0.000470,0.162974,0.003733,0.000282,0.000254,0.018976,0.000272,0.002401,0.002490,0.030871,0.000394,0.001194,0.000774,0.004808,0.073316,0.003926,0.002912,0.052293,0.207338,0.001303,0.092887,0.018312,0.001527,0.001244,0.000193,0.003151,0.000508,...,0.042068,0.010247,0.002398,0.002293,0.003522,0.009217,0.045123,0.010360,0.120103,0.002097,0.001669,0.007404,0.000276,0.000193,0.003274,0.097616,0.012205,0.001238,0.004142,0.009446,0.003323,0.008590,0.001854,0.000136,0.003733,0.041899,0.005470,0.003095,0.000691,0.000395,0.000602,0.002619,0.019650,0.000852,0.011896,0.001305,0.037324,0.011939,0.000890,0.000709
2,JMXLWAX.jpeg,0.246907,0.241164,0.011288,0.000936,0.000805,0.001521,0.000471,0.000890,0.008200,0.074137,0.000334,0.003800,0.000069,0.124202,0.000765,0.000111,0.000181,0.019144,0.000086,0.004785,0.004406,0.026109,0.000100,0.000177,0.000310,0.000224,0.006192,0.001931,0.000775,0.041116,0.226712,0.000242,0.085600,0.002732,0.000908,0.001009,0.000072,0.000699,0.000196,...,0.096381,0.002329,0.001025,0.002637,0.000659,0.000578,0.076276,0.006614,0.094081,0.000202,0.000129,0.001732,0.000090,0.000169,0.002150,0.070869,0.001852,0.000044,0.007819,0.010276,0.000516,0.000569,0.000159,0.000038,0.000765,0.033129,0.000859,0.001578,0.001429,0.000279,0.000494,0.000829,0.007589,0.000070,0.011055,0.000621,0.073624,0.007117,0.000839,0.000235
3,CV8FWT2.jpeg,0.243212,0.202613,0.036235,0.005761,0.003844,0.010522,0.001825,0.002721,0.037575,0.022500,0.004207,0.001274,0.000214,0.033990,0.001943,0.000428,0.000469,0.086723,0.001613,0.017218,0.002034,0.031783,0.000704,0.000254,0.000231,0.006729,0.017762,0.005041,0.011

In [15]:
drive.flush_and_unmount()

In [16]:
end_time = time.time()
print("Runtime of the Notebook: {} min".format(np.round((end_time - start_time) / 60, 2)))

Runtime of the Notebook: 92.04 min
